In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")


In [4]:
from langchain_core.messages import HumanMessage
llm.invoke([HumanMessage(content="Hi, My name is Adarsha")])

AIMessage(content="Hi Adarsha, it's nice to meet you! 😊  What can I do for you today? \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0d26955e-694a-4521-b5c0-c49a2fa51148-0', usage_metadata={'input_tokens': 9, 'output_tokens': 23, 'total_tokens': 32})

In [19]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(llm,get_session_history)

In [27]:
config={"configurable": {"session_id": "chat1"}}

In [28]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is Adarsha")],
    config=config,
)


In [29]:
response.content

"Hi Adarsha! It's nice to meet you. 😊  Is there anything I can help you with today? \n"

In [30]:
with_message_history.invoke(
    [HumanMessage(content="What's My Name?")],
    config=config,
)

AIMessage(content='Your name is Adarsha! 😊  I remember that from our previous interaction. 😄 \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-10f5aefd-2aa3-4049-a1c0-f7d2931d8c7b-0', usage_metadata={'input_tokens': 108, 'output_tokens': 18, 'total_tokens': 126})

In [ ]:
### If config is changed it will not remeber previous conversations

In [38]:
### Prompt Templates

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are an Ccomplaint registering CHatbot on behalf of IRTC. Answer all question to your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | llm

In [33]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Adarsha")]})

AIMessage(content='Hello Adarsha, welcome to the IRTC complaint registration system. How can I help you today? \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-b99534f8-e884-441d-bbe7-0e2f6192f60f-0', usage_metadata={'input_tokens': 22, 'output_tokens': 21, 'total_tokens': 43})

In [34]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [37]:
config ={"configurable": {"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="My name is Adarsha")],
    config=config
)

In [40]:
response = chain.invoke({"messages":[HumanMessage(content="My name is Adarsha")],"language":"Hindi"})
response.content

'नमस्ते, आदर्श जी! आपका स्वागत है। कृपया मुझे बताएं कि मैं आपकी कैसे मदद कर सकता हूं? \n'